# Hamiltonian Simulation

Quantum Hamiltonian simulation is one of the most important problems in quantum computing. It consists in modeling the time evolution of a quantum system, ruled by some not natively executable Hamiltonian in a specific device. Simulations of this type are essential to comprehend quantum systems that are described by a complex dynamics, such as molecules in chemistry or materials in condensed matter physics, where classical computers fail due the exponential scale up required by larger quantum systems.

In this guide, you will be using Classiq to work with simple problems of Hamiltonian simulation using two different methods:

* **Trotter-Suzuki decomposition**
* **qDRIFT**

# Table of contents

1. [Importance](#Importance)
2. [Hamiltonian Simulation Methods](#hamiltonian-simulation-methods)
    * [Trotter-Suzuki decomposition](#trotter-suzuki-decomposition)
    * [qDRIFT](#qDRIFT)
3. [A simple example](#a-simple-example)
    * [Trotter-Suzuki Example](#trotter-suzuki-example)
    * [qDRIFT Example](#qdrift-example)
    * [Exponentiation with depth constraint](#exponentiation-with-depth-constraint)
4. [Comparison](#comparison)
5. [Summary and Exercises](#summary-and-exercises)


# Importance

Quantum Hamiltonian simulation has several significant applications. Some of them are particularly interesting for physicists:

* Quantum Chemistry: It helps in predicting properties of molecules using quantum methods such as the Quantum Variational Eigensolver (VQE), which can be more efficiently than classical methods in certain conditions.
* Materials Science: It aids in designing new materials with desired properties by simulating their quantum mechanical behavior.
* Optimization and Machine Learning: Quantum simulations can be used to solve complex optimization problems and improve machine learning algorithms.

Mathematically, we can approach the Hamiltonian Simulation problem as the following: Suppose we have a quantum computer that can run natively the evolution operators of the set of Hamiltonians $\{H_j\}$. Now, suppose we want to understand the dynamics of the following, more complex, Hamiltonian:

<details><summary>What are the operations that this quantum computer can effectively execute?</summary>

If this quantum computer can only execute the evolution operators of the set of Hamiltonians $\{H_j\}$, then we could have only operations in the form:

$$O = \prod_{j} e^{-i \,\tau_j H_j}.$$
</details>


\begin{equation}
H_{full} = \sum_{j} h_j H_j,
\end{equation}

For some real values $h_j$. Therefore, we would need some kind of decomposition from the operator of Eq. (1) into our set of operations, formed by $\{e^{-itH_j}\}$, where $e^{-itH_j}$ is the evolution operator of the hamiltonian $H$.

# Hamiltonian simulation methods

This section provide a short introduction for two Hamiltonian simulation methods: The Trotter-Suzuki and the qDRIFT.

## Trotter-Suzuki decomposition

Now that the problem is stated, a possible solution is the Trotter-Suzuki decomposition, also known as Trotterization. Simply put, this method involves "breaking" the evolution operator into the evolution operators of its components. In the first order, it looks like this:

\begin{equation*}
e^{-itH}=\exp\left\{-it\sum_j h_j H_j\right\} \approx \left(\prod_{j} e^{-it, h_j, H_j/n}\right)^n + \mathcal{O}(t^2/n).
\end{equation*}

More details about this decomposition can be found in the Mathematical Description or in the [original paper](https://arxiv.org/abs/math-ph/0506007).

This formula is very important and has applications in several scenarios. However, if we need an error that scales better than $t^2$, it is possible to achieve higher-order Trotter-Suzuki formulas. For example, the second order Trotter-Suzuki would look like this:

\begin{equation*}
    e^{-it(H_1+H_2)}=\exp\left\{-i(t_1\,H_1+t_2\,H_2)\right\} \approx \left(e^{-it_1\, H_1/(2n)}e^{-it_2\, H_2/n}e^{-it_1\, H_1/(2n)}\right)^n + \mathcal{O}(t^3/n).
\end{equation*}

Higher orders for the Trotter-Suzuki formula can be found in [[1](#ts_paper)]. However, as the number of terms in the hamiltonian and the approximation order increases, it gets harder to construct the Trotter-Suzuki formulas. For this end, we can use Classiq's function `suzuki_trotter()` to execute a Trotter decomposition of an operator, specifying the order and number of repetitions. Lets take a look on how does it work with an example:

## qDRIFT

The quantum stochastic drift protocol (qDRIFT) is similar to the first order Trotter-Suzuki decomposition, however it is based on a stochastic distribution of the evolution operators:  The algorithm works by sampling unitaries from the set $\{e^{-itH_j}\}$, according to a probability distribution defined by the weights $\{h_j\}$, normalized by a factor $\lambda = \sum_j h_j$. In other words, the qDRIFT protocol works as:

**Input**: A list of Hamiltonian terms $\{Hj\}$, a classical oracle function SAMPLE() that returns an value $j$ according to the probability distribution $p_j = h_j/\lambda$ and a target precision $\epsilon$.

**Output**: An ordered list of evolution operators from the set $\{e^{-itH_j}\}$, that approximate the unitary $e^{-it\,H}$, with bound error $\epsilon$.

* Define $\sum_j h_j = \lambda$
* Set $N = \lceil 2\lambda^2t^2/\epsilon \rceil$
* Generate the ordered list of evolution operators according to the probability distribution.

The qDRIFT has been proved to be a good alternative for the Trotter-Suzuki decomposition when the number of terms in the Hamiltonian is not sparse, i.e., the number of terms on the expansion $H = \sum_j h_j H_j$ is not small when compared to all possible terms in it. This interesting method can be applied using Classiq's function ``qdrift()``. Its inputs are:

* ``pauli_operator``: ``CArray[PauliTerm]``: A list of Pauli operators that should be representing the terms $H_j$ and its respective coefficients $h_j$;
* ``evolution_coefficient ``: ``CArray`` The value $t$ for the coefficient in the time evolution operator;
* ``num_qdrift``:``CInt``: The number $N$ of unitary operators in the list of gates given by the qdrift;
* ``qbv``:``QArray[QBit]``: The target qubits.

# A simple example

Now that the inputs for the different methods are defined, lets apply it to the simple example of approximating the dynamics of the following two-qubits hamiltonian 


$$H = 0.3 \, X\otimes Y + 0.7 \,Y\otimes Z + 0.2\, Z\otimes X.$$

First, we need to identify the operators $\{H_j\}$ and its respective coefficients $\{h_j\}$:

$$
\begin{split}
    H_1 = X \otimes Y,\text{ coefficient: }h_1 = 0.3; \\
    H_2 = Y \otimes Z,\text{ coefficient: }h_2 = 0.7; \\
    H_3 = Z \otimes X,\text{ coefficient: }h_3 = 0.2. \\
\end{split}
$$

The second step is to define the number $N_{\text{operators}}$ of unitary operators in the list. In this case, we will apply it for $N_{\text{operators}}=30$, which would be the equivalent of having $N_{\text{repetitions}}=10$ repetitions in the Trotter-Suzuki decomposition. Before writing a program for each method, we will define the terms and coefficients of $H$ in Classiq:

In [2]:
from classiq import *
H = [       PauliTerm(pauli=[Pauli.X, Pauli.Y], coefficient=0.3),
            PauliTerm(pauli=[Pauli.Y, Pauli.Z], coefficient=0.7),
            PauliTerm(pauli=[Pauli.Z, Pauli.X], coefficient=0.7)]

Now, its time to turn the attention to the execution of the different methods:

## First order Trotter-Suzuki Example

In [5]:
from classiq import *
import numpy as np
nqubits = 2
random_initial_state = np.random.rand(2**nqubits)
random_initial_state = random_initial_state/sum(random_initial_state)
probs = random_initial_state.tolist()
@qfunc
def main(qba: Output[QArray[QBit]]):
    prepare_state(probabilities = probs, bound = 0.01, out=qba)
    suzuki_trotter(
        [
            PauliTerm(pauli=[Pauli.X, Pauli.Y], coefficient=0.3),
            PauliTerm(pauli=[Pauli.Y, Pauli.Z], coefficient=0.7),
            PauliTerm(pauli=[Pauli.Z, Pauli.X], coefficient=0.7),
        ],
        evolution_coefficient=1.0,
        order=1,
        repetitions=3,
        qbv=qba,
    )


qmod = create_model(main)
qprog = synthesize(qmod)
write_qmod(qmod, "trotter")
show(qprog)

results_trotter = execute(qprog).result()


Opening: https://platform.classiq.io/circuit/6175d7a2-cc16-4970-9660-d307c20afaeb?version=0.43.1


## qDRIFT Example

In [6]:
from classiq import *


@qfunc
def main(qba: Output[QArray[QBit]]):
    prepare_state(probabilities = probs, bound = 0.01, out=qba)
    qdrift(
        [
            PauliTerm(pauli=[Pauli.X, Pauli.Y], coefficient=0.3),
            PauliTerm(pauli=[Pauli.Z, Pauli.I], coefficient=0.7),
            PauliTerm(pauli=[Pauli.Z, Pauli.I], coefficient=0.2),
        ],
        evolution_coefficient=1.0,
        num_qdrift=9,
        qbv=qba,
    )



qmod = create_model(main)
write_qmod(qmod, "qdrift")
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/548399ec-6f47-4f5d-96aa-419ee1e5dbe0?version=0.43.1


In Classiq, it is possible to see how deep becomes the model after synthesizing. For this, it is possible to check the IDE information, in the left panel:

## Exponentiation with depth constraint

It is also possible to generate an efficient decomposition of an evolution operator with Classiq's function ``exponentiation_with_depth_constraint``. Given the maximum depth of the decomposition, and the inputs related to the Hamiltonian itself, the synthesizer finds the most accurate higher order Trotter decomposition and apply it. For the example above, considering a maximum depth of $N=30$ the code would be:

In [7]:
from classiq import *

@qfunc
def main(qba: Output[QArray[QBit]]):
    prepare_state(probabilities = probs, bound = 0.01, out=qba)
    exponentiation_with_depth_constraint(
        [
            PauliTerm(pauli=[Pauli.X, Pauli.Y], coefficient=0.3),
            PauliTerm(pauli=[Pauli.Y, Pauli.Z], coefficient=0.7),
            PauliTerm(pauli=[Pauli.Z, Pauli.X], coefficient=0.2),
        ],
        evolution_coefficient=1,
        max_depth=30,
        qbv=qba,
    )


qmod = create_model(main)
write_qmod(qmod, "exponentiation")
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/b437f4e6-8bae-4b18-89ce-4ce1c117fe1e?version=0.43.1


# Comparison

Now, its time to see how did these different methods perform in comparison with the exact evolution. For this, the exact evolution of the Hamiltonian will be done using exponentiation through the following code:

# Summary and Exercises

# References

<a id='ts_paper'>[1]</a>: [Finding Exponential Product Formulas of Higher Orders (Naomichi Hatano and Masuo Suzuki)](https://arxiv.org/abs/math-ph/0506007)